# Using Google search keywords to predict energy price volatiltiy

This notebook provides a summary and the step-by-step execution of [Google search keywords that best predict energy price volatility](https://www.sciencedirect.com/science/article/pii/S0140988317302517). Please see the paper's first section for a thorough introduction and background of the questions addressed. I have made the complete code also available in the same Github repository.

## Abstract

Internet search activity data has been widely used as an instrument to approximate trader attention in dif- ferent markets. This method has proven effective in predicting market indices in the short-term. However, little attention has been paid to demonstrating search activity for keywords that best grab investor attention in different markets. This study attempts to build the best practically possible proxy for attention in the market for energy commodities using Google search data. Specifically, we confirm the utility of Google search activity for energy related keywords are significant predictors of volatility by showing they have incremental predictive power beyond the conventional GARCH models in predicting volatility for energy commodities’ prices. Starting with a set of ninety terms used in the energy sector, the study uses a multistage filtering pro- cess to create combinations of keywords that best predict the volatility of crude oil (Brent and West Texas Intermediate), conventional gasoline (New York Harbor and US Gulf Coast), heating oil (New York Harbor), and natural gas prices. For each commodity, combinations that enhance GARCH most effectively are established as proxies of attention. The results indicate investor attention is widely reflected in Internet search activities and demonstrate search data for what keywords best reveal the direction of concern and attention in energy markets.

## Introduction
One of the most commonly accepted explanations of the observed patterns of volatility is that volatility is proportional to the rate of information inflows and investor attention. Relying on this theory, many previous studies have relied on using Google Search Volume data as a proxy for retail investor attention. The figure below illustrates  the comovement of energy price volatility and Google Search Volume data. 
![fig2](GSVimg/fig2.png)
The question we are seeking to answer is what proxies (i.e., which keywords) best capture the investor attention in the energy market.

## Data
We require two sets of data. Google Search volume and Energy commodity prices.

#### Google Search Volume
Google search time series data are downloaded for each keyword *separately* from [Google Trends](https://trends.google.com). The frequency of the data is weekly and the time window is Jan 2004 - July 2016.
The energy related keywords provided in Table 1.
![table1](GSVimg/table1.png)

#### Energy market data
Daily and weekly spot prices for crude oil (Brent and West Texas Intermediate), conventional gasoline (New York Harbor and US Gulf Coast), heating oil (New York Harbor), and natural gas for the weeks ending in Friday are downloaded from the Energy Information Administration (EIA) website. Returns and annualized weekly volatilities are calculated and the data are then combined into a single data frame.


In [ ]:
df <- read.csv("PricesAd.csv")
df$Date <- as.Date( as.character(df$Date), "%d-%b-%y")

df <- subset(df, as.Date("2004-01-01") < Date & Date < as.Date("2016-07-24"))

#returns
returns <- diff(as.matrix(log(df[,-1])))
returns <- data.frame(df$Date[-1], returns)
colnames(returns)[1] <- c("Date")

#weekly volatilites
library(xts)
colSd <- function (x, na.rm=FALSE) {apply(X=x, MARGIN=2,
                                          FUN=sd, na.rm=na.rm)}
returns <- as.xts(returns[, 2:ncol(returns)], order.by = as.Date(returns$Date))
volatilities <- apply.weekly(returns, colSd)
volatilities <- data.frame(Date=index(volatilities),
                           coredata(volatilities))

#merge with trends data
trends <- read.csv("trends.csv")
df <- cbind(volatilities, trends[,-1])

## Methodology

The methodology consists of two major parts. At first, we refine the set of keywords to keep only the terms that both Granger cause the volatility of prices and have incremental predictive power beyond the conventional GARCH model. Second, we build proxies for attention using combinations of these keywords that have predictive power beyond models using fewer keywords and have an improved adjusted-$R^2$ compared to other models. This allows us to create proxies that best explain volatility and are thus suitable proxies for attention.

#### Granger Causality

For the GSV of a keyword to be a true representative of at least some investors’ attention, it needs to be verified that the GSV leads volatility changes. This can be examined using the Granger Causality test. However, before proceeding with conducting the test, we should make sure that our time series are not unit-root processes. ADF test is used for this purpose.

In [ ]:
library(tseries)
suppressWarnings(apply(volatilities[2:ncol(volatilities)], 2, adf.test))

Based on the ADF results, the null hypothesis that a unit-root is present is rejected for all six volatility series at $1\%$ significance level. Note that although the null is not rejected for all GSV series, we are still capable of running the Granger causality test as in each test one of the variables is stationary. For the keywords and weekly volatiltiy data, the following vector autoregression models are constructed:
$$
V_t = c+ \sum_{i=1}^{p}\beta_{1i}V_{t-i} + \sum_{j=1}^{q} \beta_{2j}G_{t-j} + \epsilon_t
$$
Lag length is set to 2 for all models. The null hypothesis ($H_0$) that $GSV_t$ does not Granger Cause $V_t$ is tested using the F-test. In other terms:
$$
H_0: \beta_{2j}=0 \quad j=1,2,...,q
$$
The rejection of null-hypothesis for keyword y indicates $G_{y,t}$ can be considered to Granger cause $V_t$.

In [ ]:
library(lmtest)
GrangerTable <- matrix(NA, nrow = ncol(trends)-1, ncol = ncol(volatilities)-1)
for (i in 1:nrow(GrangerTable)){
  for (j in 1:ncol(GrangerTable)){
    pval = grangertest(df[,j+1] ~ df[,i+ncol(GrangerTable)+1], order=2, data=df)$Pr[2]
    GrangerTable[i,j] = ifelse(pval < 0.05, signif(pval, digits = 2), "--")
    #GrangerTable[i,j] = grangertest(df[,j+1] ~ df[,i+ncol(GrangerTable)+1], order=2, data=df)$Pr[2]
  }
}
colnames(GrangerTable) <- colnames(volatilities)[-1]
rownames(GrangerTable) <- colnames(trends)[-1]

#### Deriving the GARCH models
The Garch models are set-up according to the two step approach of Engle and Sheppard. This method circumvents the difficulties caused by dimensionality and allows for hypothesis testing using ordinary methods. With $a_t = r_t − \mu_t$ being the return innovation of week $t$ and letting $a_t$ follow a GARCH(1,1) process, we have $$a_t=\sqrt{h_t}\epsilon_t,$$ where $h_t$ is a process such that 
$$
h_t=\omega+\gamma a_{t-1}^2+\beta h_{t-1}
$$
This equation is estimated using the method of maximum likelihood with Student’s t-distributed errors to take into account the excess kurtosis.

In [ ]:
library(fGarch)
WeeklyRet <- apply.weekly(returns, mean)
#GARCH (1,1) model

GarchEst <- matrix(NA, nrow = 2*ncol(WeeklyRet), ncol = 7)

#Residual and cond. var. extraction
Params <- matrix(NA, nrow = nrow(WeeklyRet), ncol = 2*ncol(WeeklyRet))
for (i in 1:ncol(WeeklyRet)){
  fitted <- garchFit(formula = ~garch(1,1), data = WeeklyRet[,i], cond.dist="std")
  GarchEst[2*i-1, 1] = colnames(WeeklyRet)[i]
  GarchEst[2*i-1, 2:5] = fitted@fit$coef[1:4]
  GarchEst[2*i, 2:5] = paste("(",fitted@fit$tval[1:4], ")")
  GarchEst[2*i-1, 6] = fitted@fit$llh
  GarchEst[2*i-1, 7] = fitted@fit$ics[1]
  Params[, 2*i-1] = log(residuals(fitted)^2)
  Params[, 2*i] = fitted@h.t*1000
  #Extracting residuals and conditional variance
}

GarchEst[is.na(GarchEst)] <- ""
colnames(GarchEst) <- c("Commodity", "mu", "omega", "alpha", "beta", "logL", "AIC" )
#xtable(GarchEst)
colnames(Params) <- c(1:ncol(Params))

for (i in 1:ncol(WeeklyRet)){
  colnames(Params)[2*i-1] <- paste0("s_", colnames(WeeklyRet)[i], sep='')
  colnames(Params)[2*i] <- paste0("v_", colnames(WeeklyRet)[i], sep='')
}

df <- cbind (Params[-1,], trends[-ncol(trends),]) #using lagged trends

library(dplyr)
df <- df %>%
  select(Date, everything())

Table below shows the GARCH model estimates
![table3](GSVimg/table3.png)

#### Improving predictive power beyond GARCH

In the second step, for each commodity, the vector of conditional variances, $h_t$ is extracted from the GARCH(1,1) models to be used as the explanatory variable along with the GSV series in the following Ordinary Least Squares regressions for each keyword $y$:
$$
\ln(a_t^2) = \beta_0 + \beta_1h_{t-1} + k_1 G_{t-1} + z_t
$$
Newey and West robust standard errors account for any heteroskedasticity and autocorrelation in the residuals ($z_t).
The next step of the filtration process is performed by utilizing the developed GARCH model. For all keywords, we test the null hypothesis that the keyword’s GSV has no predictive power beyond GARCH using an F-test. Keywords for which the null is rejected are kept in the set.

#### Combining the terms to create a better proxy
Similar to the previous stage of the filtration process, in this level, we use F-tests to see whether adding GSV series for an additional keyword enhances our predictive power of shocks beyond the models. We begin by repeating the second step of deriving the GARCH model but this time adding two predictor variables instead of only one:

$$
\ln(a_t^2) = \beta_0 + \beta_1h_{t-1} + k_1 G_{1,t-1} + k_2 G_{2,t-1} + z_t
$$

Fixing the first keyword for all equations we test the hypothesis that adding GSV of one more keyword remaining in the set provides no more predictive power. The null is $k_2 = 0$. Combinations of keywords for which this hypothesis is rejected at $5\%$ significance, and whose OLS parameter estimates are significant at $5\%$ level pass this stage of filtration. These combinations are considered as predictors with predictive power beyond models that include GSV for only one keyword.

The next stage of the filtration process is exactly the same as the previous stage, and we test  the null of $k_3 = 0$ in the equation below:

$$
\ln(a_t^2) = \beta_0 + \beta_1h_{t-1} + k_1 G_{1,t-1} + k_2 G_{2,t-1} + k_3 G_{3,t-1} + z_t
$$

To the developed models, new keywords are added in a similar fashion until one of the stopping conditions is met. That is either the model fails to enhance the predictive power or there is no improvement in the adjusted R-squared.

In [ ]:
#F-test and tables 4-5-6
library(lmtest)
library(sandwich)
onelist <- list()
twolist <- list()
threelist <- list()
k1 = k2 = k3 = 1
for (i in 1:ncol(WeeklyRet)){
  z1 <- lm(df[,2*i] ~ df[,2*i+1], data = df) #original regression
  sumz1 <- summary(z1)
  err1 <- coeftest(z1, vcov = vcovHC(z1, type="HC1")) #White Errors
  for (j in 2:ncol(trends)){
    if (GrangerTable[j - 1, i]!= "--"){ #Granger causality condition
      z2 <- lm(df[,2*i] ~ df[,2*i + 1] + df[,2*ncol(WeeklyRet) + j], data = df)
      sumz2 <- summary(z2) #new equation
      err2 <- coeftest(z2, vcov = vcovHC(z2, type="HC1"))
      if (anova(z1, z2)$Pr[2] <= 0.05 && err2[3,4] <= 0.05
          && sumz2$adj.r.squared > 1.1* sumz1$adj.r.squared){ #enhancement requirements
        dat1 = c(colnames(WeeklyRet)[i], colnames(trends[j]),
                signif(sumz2$coefficients[1:3,1], 3),
                signif(sumz2$coefficients[1:3,3], 3),
                signif(sumz2$adj.r.squared, 3)) 
        onelist[[k1]] = dat1 #append
        k1 = k1 + 1
        for (l in (j+1):ncol(trends)){ #two keywords
          z3 <- lm(df[,2*i] ~ df[,2*i+1] +
                     df[,2*ncol(WeeklyRet) + j] + df[,2*ncol(WeeklyRet) + l] ,
                   data = df)
          sumz3 <- summary(z3)
          err3 <- coeftest(z3, vcov = vcovHC(z3, type="HC1"))
          if (anova(z2, z3)$Pr[2] <= 0.05 && err3[4,4] <= 0.05
              && sumz3$adj.r.squared > 1.1* sumz2$adj.r.squared){ 
            dat2 = c(colnames(WeeklyRet)[i], colnames(trends[j]),
                     colnames(trends[l]),
                    signif(sumz3$coefficients[1:4,1], 3),
                    signif(sumz3$coefficients[1:4,3], 3),
                    signif(sumz3$adj.r.squared, 3)) 
            twolist[[k2]] = dat2
            k2 = k2 + 1
            if (l < ncol(trends)){
              for (m in (l+1):ncol(trends)){
                z4 <- lm(df[,2*i] ~ df[,2*i+1] +
                           df[,2*ncol(WeeklyRet) + j] + df[,2*ncol(WeeklyRet) + l] +
                           df[,2*ncol(WeeklyRet) + m], data = df)
                sumz4 <- summary(z4)
                err4 <- coeftest(z4, vcov = vcovHC(z4, type="HC1"))
                if (anova(z3, z4)$Pr[2] <= 0.05 && err4[5,4] <= 0.05
                    && sumz4$adj.r.squared > 1.1* sumz3$adj.r.squared){
                  imp=(100*(sumz4$adj.r.squared-sumz1$adj.r.squared))/sumz1$adj.r.squared
                  dat3 = c(colnames(WeeklyRet)[i], colnames(trends[j]),
                           colnames(trends[l]),
                           colnames(trends[m]),
                           signif(sumz4$coefficients[1:5,1], 3),
                           signif(sumz4$coefficients[1:5,3], 3),
                           signif(sumz4$adj.r.squared, 3),
                           #imp)
                           round(imp, 3)) 
                  threelist[[k3]] = dat3
                  k3 = k3 + 1
                }
            }
            }
          }
        }
      }
    }
  }
}

OneKey <- data.frame(matrix(unlist(onelist), nrow=length(onelist), byrow=T))
colnames(OneKey) <- c("Commodity", "Term", "beta_0", "beta_1", "k_1",
                      "t_beta_0", "t_beta_1", "t_k_1", "Adj.R.Sq")
#OneKey <- OneKey[order(OneKey$Adj.R.Sq,decreasing = TRUE),]

TwoKey <- data.frame(matrix(unlist(twolist), nrow=length(twolist), byrow=T))
colnames(TwoKey) <- c("Commodity", "Term1", "Term2", "beta_0", "beta_1", "k_1", "k_2",
                      "t_beta_0", "t_beta_1", "t_k_1", "t_k_2", "Adj.R.Sq")
#TwoKey <- TwoKey[order(TwoKey$Adj.R.Sq,decreasing = TRUE),]

ThreeKey <- data.frame(matrix(unlist(threelist), nrow=length(threelist), byrow=T))
colnames(ThreeKey) <- c("Commodity", "Term1", "Term2", "Term3", "beta_0", "beta_1",
                      "k_1", "k_2", "k_3", "t_beta_0", "t_beta_1", "t_k_1", "t_k_2",
                      "t_k_3", "Adj.R.Sq", "improvement")

ThreeKey <- ThreeKey[order(ThreeKey$Adj.R.Sq, decreasing = TRUE),]
#rm(onelist, twolist, threelist)

This able show the best combinations of keywords for predicting volatiltiy for each energy commodity.
![table6](GSVimg/table6.png)

Based on the significance of estimates and the magnitude of the adjusted $R^2$, combination of GSV of keywords presented in table below are considered as the best proxies for investor attention. The second column shows the improvement obtained as compared to the GARCH models.
![table7](GSVimg/table7.png)